In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import joblib
from sklearn.metrics import r2_score
from helper import PgxPipeline,get_outliers, DropColumnTransformer
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("datasets/pga_Rrup.csv")
outlier_indexes = get_outliers(df)
df = df.drop(outlier_indexes)
df.head()

In [ ]:
process_pipeline = PgxPipeline("PGA",df).get_pipeline()

In [ ]:
df_processed = pd.DataFrame(
    process_pipeline.transform(df),
    columns=["MW","Rrup","VS30","PGA","FT_Ters","FT_Yanal"]
)
X = df_processed.drop("PGA",axis=1)
y = df_processed["PGA"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Define the model architecture
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=5,activation="relu"))
model.add(tf.keras.layers.Dense(units=10,activation="relu"))
model.add(tf.keras.layers.Dense(units=20,activation="relu"))
model.add(tf.keras.layers.Dense(units=10,activation="relu"))
model.add(tf.keras.layers.Dense(units=5,activation="relu"))
model.add(tf.keras.layers.Dense(units=1))
model.compile(optimizer="adam",loss="mean_squared_error")
model.fit(x_train,y_train,batch_size=32,epochs=100)

In [11]:
def create_model(num_units, num_layers, loss_function, optimizer_function):
    model = tf.keras.models.Sequential()
    for i in range(num_layers):
        model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss=loss_function, optimizer=optimizer_function)
    return model

In [13]:
param_grid = {
    'num_units': [5, 20,100],
    'num_layers': [1, 2, 3,4,5],
    'loss_function': ['mse', 'mae'],
    'optimizer_function': ['adam', 'rmsprop']
}
model = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model, verbose=0)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_result = grid_search.fit(x_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.107850 using {'loss_function': 'mse', 'num_layers': 5, 'num_units': 100, 'optimizer_function': 'adam'}


In [14]:
final_model = create_model(**grid_result.best_params_)
final_model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
468/468 [==============================] - 2s 1ms/step - loss: 0.1354
Epoch 2/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1090
Epoch 3/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1071
Epoch 4/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1058
Epoch 5/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1049
Epoch 6/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1031
Epoch 7/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1022
Epoch 8/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1017
Epoch 9/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1013
Epoch 10/100
468/468 [==============================] - 1s 1ms/step - loss: 0.1008
Epoch 11/100
468/468 [==============================] - 1s 1ms/step - loss: 0.0999
Epoch 12/100
468/468 [==============================] - 1s 1ms/step - loss: 0.0988
Epoch 13/100


In [16]:
y_predict = final_model.predict(x_test)
test_score = r2_score(y_test, y_predict)
print("test_score = ", test_score)

117/117 [==============================] - 0s 1ms/step
test_score =  0.9037590427164497
